In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from P6_code.FinishedCode.importData import ImportEV
from P6_code.FinishedCode.dataTransformation import createUsers
from P6_code.FinishedCode.functions import split_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
start, end = "2018-06-01", "2018-11-09"
df_train = ImportEV().getCaltech(start_date=start, end_date=end, removeUsers=True, userSampleLimit=25)

In [3]:
users = createUsers(df_train, start, end)
userID = users.data.userID.unique()
user_df = []

for user in userID:
    user_df.append(users.getUserData(user=user))

print(user_df[2])

      chargingTime  kWhDelivered  Year Month Day  Weekday Hour
0              0.0           0.0  2018    06  01        4   00
1              0.0           0.0  2018    06  01        4   01
2              0.0           0.0  2018    06  01        4   02
3              0.0           0.0  2018    06  01        4   03
4              0.0           0.0  2018    06  01        4   04
...            ...           ...   ...   ...  ..      ...  ...
3859           0.0           0.0  2018    11  08        3   19
3860           0.0           0.0  2018    11  08        3   20
3861           0.0           0.0  2018    11  08        3   21
3862           0.0           0.0  2018    11  08        3   22
3863           0.0           0.0  2018    11  08        3   23

[3864 rows x 7 columns]


In [4]:
max_kWh = max([user["kWhDelivered"][i] for user in user_df for i in range(len(user))])

In [5]:
max_charging = max([user["chargingTime"][i] for user in user_df for i in range(len(user))])

In [6]:
X, Y = [], []


for user in user_df:
    Y.append(user[['chargingTime', 'kWhDelivered']])
    X.append(user.drop(columns=['chargingTime', 'kWhDelivered']))
Y

[      chargingTime  kWhDelivered
 0              0.0           0.0
 1              0.0           0.0
 2              0.0           0.0
 3              0.0           0.0
 4              0.0           0.0
 ...            ...           ...
 3859           0.0           0.0
 3860           0.0           0.0
 3861           0.0           0.0
 3862           0.0           0.0
 3863           0.0           0.0
 
 [3864 rows x 2 columns],
       chargingTime  kWhDelivered
 0              0.0           0.0
 1              0.0           0.0
 2              0.0           0.0
 3              0.0           0.0
 4              0.0           0.0
 ...            ...           ...
 3859           0.0           0.0
 3860           0.0           0.0
 3861           0.0           0.0
 3862           0.0           0.0
 3863           0.0           0.0
 
 [3864 rows x 2 columns],
       chargingTime  kWhDelivered
 0              0.0           0.0
 1              0.0           0.0
 2              0.0       

In [14]:
ss = StandardScaler()

for user in X:
    ss.fit(user)

X_scaled = []

for user in X:
    X_scaled.append(ss.transform(user))


In [15]:
sc = MinMaxScaler(feature_range = (0,1))

for user in Y:
    sc.fit(user[['chargingTime', 'kWhDelivered']])

user_df_scaled = []

for user in Y:
    user_df_scaled.append(sc.transform(user[['chargingTime', 'kWhDelivered']]))

user_df_scaled

[array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.]

In [16]:
total_samples = len(X[0])
train_test_cutoff = round(0.2 * total_samples)

In [18]:
X_train, X_val = [], []
Y_train,Y_val = [], []

for user in range(len(user_df_scaled)):
    user_X, user_Y = split_sequences(X_scaled[user], user_df_scaled[user], 10, 5)

    X_train.append(user_X[:-train_test_cutoff])
    X_val.append(user_X[-train_test_cutoff:])

    Y_train.append(user_Y[:-train_test_cutoff])
    Y_val.append(user_Y[-train_test_cutoff:])

In [19]:
model = Sequential()
model.add(LSTM(4, activation='relu', return_sequences=True, input_shape=(10,5))) # todo: add inputs steps
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [41]:
#train_X = np.stack(X_train, axis=1)
#train_Y = np.stack(Y_train, axis=1)

model.fit(X_train.values, Y_train, epochs=30, verbose=0)

AttributeError: 'list' object has no attribute 'values'

In [32]:
for i in range(len(Y_train)):
    print(Y_train[i].shape)

(3078, 5)
(3078, 5)
(3078, 5)
(3078, 5)
(3078, 5)
(3078, 5)
(3078, 5)
(3078, 5)
(3078, 5)
(3078, 5)
(3078, 5)
(3078, 5)
